# Pondération des équipements de la BPE

Dans la section précédente, j’ai montré comment filtrer les données de la BPE afin de ne conserver que les équipements associés à une catégorie sociale.
J’ai également introduit la notion de pondération, qui permet de tenir compte à la fois de l’importance propre des équipements et de la diversité de leur environnement.

Dans cette partie, je vais montrer comment attribuer des poids aux équipements.


## Les types de pondération

Deux dimensions de pondération sont considérées :

1. **Pondération par importance intrinsèque** : chaque type d’équipement reçoit un poids en fonction de son rôle social. Par exemple, un supermarché peut avoir un poids plus élevé qu’une boulangerie.

2. **Pondération par variété environnante** : la valeur d’un équipement dépend aussi du contexte. Par exemple, un parc situé près de plusieurs écoles et centres communautaires peut être considéré comme plus précieux qu’un parc isolé.


Pour cela, je vais utiliser des outils d’analyse spatiale en Python, **GeoPandas**, **NumPy** et la classe **BallTree** de `scikit-learn`, qui permet de gérer les calculs de voisinage.

In [1]:
# Import bibliothèque 
import numpy as np
import geopandas as gpd
from sklearn.neighbors import BallTree

## Chargement des données

Je commence par recharger les données de la **BPE (2023)** que j’ai sauvegardées lors du prétraitement dans la section précédente.
À ce stade, le fichier contient uniquement les équipements filtrés selon leur qualité de géolocalisation et regroupés par grandes catégories sociales (santé, éducation, commerce, etc.).

Pour la suite de la démonstration, je vais m’appuyer sur l’exemple du domaine **santé**.

In [2]:
data_path = "../../proxy/data/processed/b3_2501171030_health_mrs.gpkg" 
# Chargement des données de la BPE (points)
data = gpd.read_file(data_path)
# Affichage des premières lignes du GeoDataFrame
data.head()

,TYPEQU,cat,wi,wecat,geometry
0,D201,health,0.0,0.0,POINT (901648.360 6253959.190)
1,D201,health,0.0,0.0,POINT (901650.000 6253850.000)
2,D201,health,0.0,0.0,POINT (901387.000 6251711.700)
3,D201,health,0.0,0.0,POINT (901236.730 6251635.020)
4,D201,health,0.0,0.0,POINT (901244.210 6251911.890)


## Pondération intrinsèque des équipements

L’objectif est d’associer à chaque type d’équipement un poids numérique, reflétant son importance sociale.
Ces poids sont définis de manière subjective, en fonction du rôle joué par l’équipement dans la vie quotidienne.

Pour illustrer la démarche, je reprends l’exemple du domaine **santé** et je construis un dictionnaire où les clés correspondent aux types d’équipements et les valeurs aux poids attribués.

In [3]:
poids_health = {
    'D108': 2.0, # CENTRE DE SANTÉ
    'D113': 2.0, # MAISON DE SANTÉ PLURIDISCIPLINAIRE
    'D201': 1.0, # MÉDECIN GÉNÉRALISTE
    'D233': 1.0, # MASSEUR KINÉSITHÉRAPEUTE
    'D302': 1.0, # LABORATOIRE D'ANALYSES ET DE BIOLOGIE MÉDICALE
    'D307': 3.0, # PHARMACIE
}

## Attribution des poids aux équipements

### Poids intrinsèques

La colonne `wi` est complétée automatiquement à partir du dictionnaire : chaque équipement reçoit son poids, et ceux non définis prennent la valeur `0`.

In [5]:
# Fonction pour obtenir le poids en fonction de TYPEQU
def get_weight(row):
    if row['TYPEQU'] in poids_health:
        return poids_health[row['TYPEQU']]
    else:
        return 0  # Valeur par défaut si TYPEQU n'est pas trouvé

# Application de la fonction pour remplir la colonne `wi`
data['wi'] = data.apply(get_weight, axis=1)
# Affichage des premières lignes pour vérifier les poids attribués
data.head()

,TYPEQU,cat,wi,wecat,geometry
0,D201,health,1.0,0.0,POINT (901648.360 6253959.190)
1,D201,health,1.0,0.0,POINT (901650.000 6253850.000)
2,D201,health,1.0,0.0,POINT (901387.000 6251711.700)
3,D201,health,1.0,0.0,POINT (901236.730 6251635.020)
4,D201,health,1.0,0.0,POINT (901244.210 6251911.890)


In [7]:
data['cat'].unique()

array(['health'], dtype=object)

### Poids extrinsèques (variété environnante)

La colonne `wecat` dépend de la diversité des équipements autour de chaque point.
Concrètement, je regarde combien de types différents se trouvent dans un certain rayon (un paramètre que je choisis). Plus la variété est grande, plus le poids augmente.
Pour faire ce calcul rapidement, j’utilise `BallTree` de `scikit-learn`, qui permet de trouver efficacement les voisins dans ce rayon.

In [8]:
# Construction de l'arbre spatial avec BallTree
coords = np.vstack([data.geometry.x, data.geometry.y]).T
tree = BallTree(coords, metric='euclidean')

In [10]:
# Je ne compte pas l'equipement principal, il a dejà un poids intrinsèque

rayon = 50.0 # Rayon pour calculer la diversité

def wecat(index, point, typequ, radius = rayon): 

    indices = tree.query_radius([point], r=radius)[0]
    
    neighboors = data.iloc[indices]
    types_diff = neighboors['TYPEQU'].unique()
    
    types_diff = types_diff[types_diff != typequ]
    
    return len(types_diff) #+ 1 pour compter l'equipement principale

In [11]:
# calcul et Ajout des poids extrasinque dans les données
data.loc[:, 'wecat'] = data.apply(lambda row: wecat(row.name, [row.geometry.x, row.geometry.y], row['TYPEQU']), axis=1)

In [12]:
# Affichage pour verif
data.tail()

,TYPEQU,cat,wi,wecat,geometry
4665,D307,health,3.0,2.0,POINT (887533.550 6254087.900)
4666,D307,health,3.0,1.0,POINT (889993.590 6253679.560)
4667,D307,health,3.0,2.0,POINT (888278.580 6254291.320)
4668,D307,health,3.0,2.0,POINT (889128.080 6253952.660)
4669,D307,health,3.0,1.0,POINT (889005.540 6254025.890)


## Sauvegarde des données pondérées

Les données, désormais enrichies avec les poids, sont sauvegardées dans un nouveau fichier GeoPackage afin de pouvoir être réutilisées dans les analyses suivantes.

In [13]:
data.to_file("../../proxy/data/processed/b3_2501171030_health_mrs_50.gpkg", driver="GPKG")

## Conclusion

Dans cette section, j’ai présenté la démarche de pondération des équipements.
D’abord, chaque équipement a reçu un **poids intrinsèque**, en fonction de son importance sociale. Ensuite, j’ai ajouté un **poids extrinsèque**, basé sur la diversité des équipements présents dans son voisinage.

Le résultat est un GeoDataFrame enrichi et sauvegardé dans un fichier GeoPackage.
Cette base pondérée constitue un point d’appui pour la suite du travail, et sera utilisée dans le calcul des indices de proximité dans les prochaines sections.